In [1]:
import torch
torch.__version__

'2.3.1+cu121'

In [4]:
with open('input.txt','r', encoding='utf-8') as f:
    text = f.read()

In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)


inputs:
torch.Size([4, 8])
tensor([[31, 43, 56, 60, 39, 52, 58, 10],
        [59, 56,  1, 44, 53, 43,  8,  0],
        [ 0,  0, 19, 30, 33, 25, 21, 27],
        [58, 46,  1, 61, 53, 43,  8,  0]])
targets:
torch.Size([4, 8])
tensor([[43, 56, 60, 39, 52, 58, 10,  0],
        [56,  1, 44, 53, 43,  8,  0,  0],
        [ 0, 19, 30, 33, 25, 21, 27, 10],
        [46,  1, 61, 53, 43,  8,  0,  0]])


In [18]:
from torch import nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets is a tensor of shape (batch_size, seq_len) or (B,T)
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets==None:
            loss = None
        else:
            B, T, C = logits.shape
            # reshaping for calculating the loss
            logits = logits.view(-1, C) # or (B*T, C)
            targets = targets.view(-1) # or (B*T,)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_tokens):
        # idx is a tensor of shape (batch_size, seq_len) or (B,T)
        for _ in range(max_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [21]:
m = BigramModel(vocab_size)
logits, loss = m(xb, yb)
print(loss.shape)
print(loss)

res = m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_tokens = 100)
print(decode(res[0].tolist()))

torch.Size([])
tensor(4.5247, grad_fn=<NllLossBackward0>)

,E&!&N$WatTc3?qSHTpHCki3s
pamUPNXfrnf&!tWvz?vn&NrPiCROWJngHpt!h
kDSHw. iw.XXMTcOek
,$eyUGbwECZhwFNFH


In [25]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32

for i  in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f'step: {i}  loss: {loss.item()}')
print(loss.item())


step: 0  loss: 2.4623334407806396
step: 1000  loss: 2.5555975437164307
step: 2000  loss: 2.4037024974823
step: 3000  loss: 2.4166741371154785
step: 4000  loss: 2.400641679763794
step: 5000  loss: 2.43243670463562
step: 6000  loss: 2.5376839637756348
step: 7000  loss: 2.4350857734680176
step: 8000  loss: 2.4135007858276367
step: 9000  loss: 2.4512810707092285
2.4195501804351807


In [27]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx,200)[0].tolist()))


HARThesma?
Whelknthistlur Rorate.'s h wavel wsave he;
Thendse nd a if amisoupl nlls it,
Grer IULAncidenare ordimyse oomaulitus efo wo'exWhes darken alllf e y se ht laly.

ICHENICI k?
ES: ho t 'Pre h p


## Mathamatical trick of self-attention
- here we will be using matrix multiplication to find the average
- input matrix will have an extra element in each line

In [26]:
torch.manual_seed(42)
a = torch.ones(3,3)
a = torch.tril(a)
a = a/torch.sum(a,1,keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [38]:
B,T,C = 4,8,2 # batch, time, channels
a = torch.tril(torch.ones((T,T)))
a = a.masked_fill(a==0, float('-inf') )
a = a.masked_fill(a==1, 0 )
print(a,"\n\n")
print(F.softmax(a))

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]]) 


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


C:\Users\tasli\AppData\Local\Temp\ipykernel_16756\3887107582.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(a))


### Attention = softmax( Q*K / sqrt(d))
- Q- query
- k- key
- d- dimension
- division is done for normalization of softmax
- softmax aggregates values to when extrem values occure

In [41]:
F.softmax(torch.Tensor([.1,.2,-.1,1,.5]))

C:\Users\tasli\AppData\Local\Temp\ipykernel_16756\1663405474.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.Tensor([.1,.2,-.1,1,.5]))


tensor([0.1454, 0.1607, 0.1191, 0.3577, 0.2170])

In [43]:
F.softmax(torch.Tensor([.1,.2,-.1,.5])*8)

C:\Users\tasli\AppData\Local\Temp\ipykernel_16756\1499388412.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.Tensor([.1,.2,-.1,.5])*8)


tensor([0.0358, 0.0796, 0.0072, 0.8774])

to avoid this aggregation to large value we normalise by dividing by sqrt(d)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

## self attention head example

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
lr= 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
eval_iters = 200
n_embd = 32
dropout = 0.0
torch.manual_seed(1337)



C:\Users\tasli\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\cuda\__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [16]:
with open('input.txt','r') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    return x, y

decode(get_batch('train')[0][0].tolist())

'revived, and was an emperor.\nAh '

In [54]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X,y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    return out

In [61]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

m = MultiHeadAttention(n_head, 4)

In [62]:
    
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
         # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size) # sa stands for self attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
     
    def forward(self,x):
        x = x+ self.sa(self.ln1(x)) #actually we aare giving a direct connection with the result to 
        x = x + self.ffwd(self.ln2(x))
        return x
    
b = Block(n_embd, n_head)

In [65]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self,idx,targets=None):
        B,T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,Vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss  = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


0.055745 M parameters


In [66]:



optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for iter in range(max_iters):
    if iter%eval_interval==0 or iter ==max_iters-1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch("train")

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.3764, val loss 4.3753
step 100: train loss 3.0383, val loss 3.0575
step 200: train loss 2.6939, val loss 2.6879
step 300: train loss 2.5844, val loss 2.5859
step 400: train loss 2.5240, val loss 2.5256
step 500: train loss 2.4772, val loss 2.4770
step 600: train loss 2.4278, val loss 2.4408
step 700: train loss 2.4087, val loss 2.4077
step 800: train loss 2.3707, val loss 2.3839
step 900: train loss 2.3348, val loss 2.3408
step 1000: train loss 2.3171, val loss 2.3227
step 1100: train loss 2.2898, val loss 2.3018
step 1200: train loss 2.2750, val loss 2.2848
step 1300: train loss 2.2454, val loss 2.2532
step 1400: train loss 2.2344, val loss 2.2376
step 1500: train loss 2.2230, val loss 2.2399
step 1600: train loss 2.1887, val loss 2.2175
step 1700: train loss 2.1908, val loss 2.2159
step 1800: train loss 2.1723, val loss 2.1909
step 1900: train loss 2.1550, val loss 2.1832
step 2000: train loss 2.1440, val loss 2.1637
step 2100: train loss 2.1301, val loss 2.1659


In [69]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


He thard sory a way grean no is him.

BEUTIOLA:
You all wareath, thenercedd,
Con. wels I my lood:
Pery, alandiobly
Wils, be jotck che endowen: of gome thelst and tickes a scerelfs.

COF YORK PUTHER:
This bauct do and have besen than 'ou my,
Ip fard Kown he ladied to do tredas?

Ficer Ray-tabliete geverts,
Is hath the of juckess the belod:
Haply goen to poor.

MANINUS:
Give you Hourrders this eserar?
Haved us any min sirre comblester.

DUSTARMUNI:
Fello, stall Edpok Ramos, in is hee geratoue,
A ofoxedine him with good be he.

DUCKINGHAS, mare not therer mad hopt yeveor bech dabrie,
I to not call let this ans is the bladik
boid diss pist am, is ust maire the hat Edtherd.

CAGLUD RISCKhat. I mirrre bratintced leasser's of my loreds.
I laster but safbrat, sp goee good with.

Then atte, some.

QSow To as thee, Wit sheald ned a resont; in him
I hir had belared fort bauged pait jupor be not, ic and lean-
Dingeance worlow kinary?
Agp, ninghiond that of connott you.

Sir, is goar bestare beams